In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import re

In [3]:
df_train = pd.read_csv("../data/raw/train.csv")
df_test = pd.read_csv("../data/raw/test.csv")

In [4]:
df_train.shape, df_test.shape

((38740, 3), (26260, 2))

In [5]:
df_train

,oid,category,text
0,365271984,winter_sport,Волшебные фото Виктория Поплавская ЕвгенияМедв...
1,503385563,extreme,Возвращение в подземелье Треша 33 Эйфория тупо...
2,146016084,football,Лучшие чешские вратари – Доминик Доминатор Гаш...
3,933865449,boardgames,Rtokenoid Warhammer40k валрак решил нас подкор...
4,713550145,hockey,Шестеркин затаскивает Рейнджерс в финал Восточ...
...,...,...,...
38735,910636962,autosport,8 битная буря снова накрыла пикселями автомоби...
38736,669736851,autosport,Ира Сидоркова объясняет как сказалась на ее ма...
38737,558919241,tennis,24 я ракетка мира хорват Марин Чилич обыграл и...
38738,776944963,volleyball,Стал известен календарь мужской сборной России...


In [6]:
df_train.category.value_counts()

category
autosport       3160
extreme         3110
martial_arts    3050
motosport       3030
boardgames      3020
tennis          3000
esport          2990
athletics       2970
hockey          2950
volleyball      2950
football        2860
basketball      2850
winter_sport    2800
Name: count, dtype: int64

In [8]:
df_train.category.unique().tolist()

['winter_sport',
 'extreme',
 'football',
 'boardgames',
 'hockey',
 'esport',
 'athletics',
 'motosport',
 'basketball',
 'tennis',
 'autosport',
 'martial_arts',
 'volleyball']

In [64]:
all(df_train.oid.value_counts() == 10)

True


#### Проверим как соотносятся разные oid и их категории: правда ли что один oid может относиться только к одной категории?

In [117]:
all(df_train.groupby(["oid"]).apply(lambda x: len(x.category.unique())) == 1)

True

In [4]:
set(df_test.oid).intersection(df_train.oid)

set()

#### Колонка `oid`
- Один oid может принадлежать только одной категории. 
- Нет пересечения oid в тренировочных и тестовых выборках.
- Скорее всего `oid` - это уникальный индитификатор источника из которого был извлечен текст.

> Таким образом oid может быть использован на этапе EDA, и на этапе разделения выборки на тренировочную и валидационную, но на этапе построения модели и инференса он не нужен.

### Постановка задачи

Из условия тестовго задания:

Создать модель классификации, которая на вход принимает текст, а как результат возвращает класс. Рекомендуется использовать train.csv для тренировки, а test.csv – для теста, но не запрещается обратное.

Можно заключить по данным, что:
- класс - это колонка `category`
- колонка `oid` - скорее всего идентификатор источника, откуда был взят текст.

Таким образом, задача состоит в том, чтобы создать модель классификации, которая предсказывает принадлежит ли текст к одной из категорий:

    autosport
    extreme
    martial_arts
    motosport
    boardgames
    tennis
    esport
    athletics
    hockey
    volleyball
    football
    basketball
    winter_sport
    

# I. Поиск дубликатов.

## [1]  Поиск полных дубликатов

In [122]:
df_train[df_train.duplicated(keep="first")]

,oid,category,text
1482,842513089,basketball,Питание – один из самых важных процессов в наш...
1601,31593254,boardgames,Приветствуем коллеги 33 А во что вы играли на ...
2110,845723373,martial_arts,Это первое интервью Мокаева в Дагестане Молодо...
2130,240880977,martial_arts,Вам не нужно теперь ломать голову что подарить...
3561,885529499,martial_arts,Расписание на вторник КРАСНАЯ ПРЕСНЯ 19. 00 ММ...
...,...,...,...
38566,217746176,esport,Будет 2 победителя Принять участие Включить ув...
38607,774280988,hockey,Смотрите матч Спартак – Зенит на Кинопоиске. Д...
38651,550304346,basketball,За кроссовками в Баскетбольный магазин Ghetto ...
38668,229536361,basketball,За жилетами в Баскетбольный магазин Ghetto Bas...


In [123]:
df_train[df_train.duplicated(keep=False)].shape[0]-df_train[df_train.duplicated(keep="first")].shape[0]

341

- Всего 431 строк полных дубликатов.
- 341 строка имеет в тренировочной выборке как минимум 1 дубликат.

Удалим полные дубликаты и сохраним промежуточное состояние:

In [124]:
df_train_removed_dup = df_train.drop_duplicates(keep="first")

In [125]:
df_train_removed_dup.shape

(38309, 3)

При удалении полных дубликатов, было удалено 431 строка (1%).

## [2] Дубликаты колонки text


    Шаг 1. -  Нужно отфильтровать строки у которых различный `oid` но одинаковый `text` и `category`, оставив только одну.

    Шаг 2. -  Удалить строки у которых разная `категория`, но одинаковый `текст`. Так как это неоднозначная ситуация и без вопросов заказчику и правильной постановки бизнес задачи, разрешить эту ситуацию не получится. Скорее всего такие строки были получены "грубым" сбором данных без должной валидации качества. Оставляя такие строки в тренировочной выборке будет сложно построить модель, которая дает адекватный прогноз.

#### Шаг 1. Фильтрация дубликатов колонки text, относящихся к одной категории

In [71]:
df_train_removed_dup[df_train_removed_dup.duplicated(subset=["text", "category"], keep=False)].sort_values("text")

,oid,category,text
36144,345735095,autosport,. Фото 20 06 31. 12. 21 Привет владельцу жиги ...
8394,373777746,autosport,. Фото 20 06 31. 12. 21 Привет владельцу жиги ...
11628,675823274,extreme,. а вы знали что Эльбрус – вулкан в западной ч...
21677,871273064,extreme,. а вы знали что Эльбрус – вулкан в западной ч...
37735,839788862,winter_sport,10 летняя девочка стала самой молодой горнолыж...
...,...,...,...
32840,507721968,autosport,️Почему нельзя драпировать мавзолей Ленина оди...
35188,565049792,autosport,️Россия попала в ЗАПАДню Человеку непросто мен...
35720,911451284,autosport,️Россия попала в ЗАПАДню Человеку непросто мен...
1275,307949868,esport,️Скоро в VALORANT появится новый соревнователь...


In [97]:
df_train_removed_dup[df_train_removed_dup.duplicated(subset=["text", "category"], keep=False)].sort_values("text").to_csv("dup2.csv")

In [74]:
df_train_removed_dup2 = df_train_removed_dup.drop_duplicates(subset=["text", "category"], keep="first")
df_train_removed_dup2.shape

(35965, 3)

При удалении дубликатов по колонкам "text", "category", удалили 2344 строки (6 %).

#### Шаг 2. Полное удаление дубликатов колонки text, относящихся к разным категориям


In [85]:
df_train_removed_dup2[df_train_removed_dup2.duplicated(subset=['text'], keep=False)].sort_values("text")

,oid,category,text
9660,749181195,martial_arts,30 дней Okko бесплатно 33 Подключайтесь и смот...
7741,410518766,hockey,30 дней Okko бесплатно 33 Подключайтесь и смот...
7521,457739190,volleyball,60 дней Плюса Мульти в подарок по промокоду MY...
25576,863906059,esport,60 дней Плюса Мульти в подарок по промокоду MY...
12087,200619851,basketball,60 дней Плюса Мульти в подарок по промокоду MY...
...,...,...,...
8323,89193385,autosport,﻿Пришла беда откуда не ждали? ️Государство не ...
32678,419210277,boardgames,﻿⭐️ Россия обязана победить США на Украине ина...
34873,565049792,autosport,﻿⭐️ Россия обязана победить США на Украине ина...
978,624659062,autosport,﻿⭐️ Учитель с большой буквы Сколько денег вкла...


In [87]:
df_train_removed_dup3 = df_train_removed_dup2.drop_duplicates(subset=['text'], keep=False)

In [88]:
df_train_removed_dup3.shape

(35631, 3)

In [116]:
df_train_removed_dup3.to_csv("../data/interim/train_no_dup.csv")

Удалено 334 строки.